In [1]:
# from collections import Counter
import pickle
import torch
import numpy as np
# import codecs
import gensim
# import time
from gensim.models import KeyedVectors
import os
import pandas as pd
import csv
import fasttext
import sys
import torch
import crypten
crypten.init()
from math import sqrt

In [2]:
vectors_filename = "/home/jovyan/embeddings/BioWordVec_PubMed_MIMICIII_d200.vec.bin"
model_filename = "/home/jovyan/embeddings/BioWordVec_PubMed_MIMICIII_d200.bin"

#! Remove this path once I implement PSI
deid_notes_path = "/home/jovyan/mpc_use_case/three_party_mpc/party1_unstructured/data/deidentified_notes"

knee_tensors_path = "/home/jovyan/mpc_use_case/three_party_mpc/party2_structured/data/single_word_tensors/knee_oa_tensors.pth"
hip_tensors_path = "/home/jovyan/mpc_use_case/three_party_mpc/party2_structured/data/single_word_tensors/hip_oa_tensors.pth"

knee_keywords_output = "/home/jovyan/mpc_use_case/three_party_mpc/party2_structured/data/convert_data_output/knee_keywords_test.pt"
hip_keywords_output = "/home/jovyan/mpc_use_case/three_party_mpc/party2_structured/data/convert_data_output/hip_keywords_test.pt"

In [3]:
bioword_vector = KeyedVectors.load_word2vec_format(vectors_filename, binary=True)
print("Vectors loaded")

Vectors loaded


In [4]:
bioword_model = fasttext.load_model(model_filename)
print("Model loaded")

Model loaded


In [5]:

#! -----------------------------------------------------------------------------------------
#TODO Untructured data - Remove after
#! -----------------------------------------------------------------------------------------
print("\nProcessing patients and their notes based on their diagnosis")

# Fetching demographic_no for all patients from the filename of notes
files = os.listdir(deid_notes_path)
all_demographic_nos_notes = set()
oa_patients = set()
for file in files:
    demographic_no = int(file.split("-")[1].split(".")[0])
    all_demographic_nos_notes.add(demographic_no)
print("Number of patients having patient notes:", len(all_demographic_nos_notes))


Processing patients and their notes based on their diagnosis
Number of patients having patient notes: 163


In [6]:
all_demographic_nos_dxresearch = set()
oa_patients = set()

# Convert txt to csv
with open('/home/jovyan/mpc_use_case/structured_data/DxResearch.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('/home/jovyan/mpc_use_case/prototype/oaTypes/DxResearch.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

# Fetching demographic_no of total patients and OA patients from the DxResearch table
df = pd.read_csv("/home/jovyan/mpc_use_case/prototype/oaTypes/DxResearch.csv")
# df.head()
for index, row in df.iterrows():
    no = row['demographic_no']
    all_demographic_nos_dxresearch.add(no)
    if row['dxresearch_code'] == 715:
        oa_patients.add(no)
print("\nNumber of patients listed with disease code:", len(all_demographic_nos_dxresearch))
print("Number of patients listed in disease code table as having OA:", len(oa_patients))


Number of patients listed with disease code: 163
Number of patients listed in disease code table as having OA: 33


In [7]:

#! -----------------------------------------------------------------------------------------
#TODO Where PSI comes in
#! -----------------------------------------------------------------------------------------
# Deducting the demographic_no of OA patients having notes
oa_patients_with_notes = oa_patients.intersection(all_demographic_nos_notes)
#print("Number of patients having OA and notes:", len(oa_patients_with_notes))
print("Patient IDs:", sorted(oa_patients_with_notes))

Patient IDs: [4, 5, 6, 7, 8, 9, 11, 14, 18, 26, 37, 40, 54, 58, 61, 63, 64, 76, 77, 83, 94, 101, 103, 106, 110, 115, 133, 135, 148, 150, 155, 159, 162]


In [8]:
# Lists that contain keywords that we can look for in the clinical notes
knee_oa_substrings = ['knee pain', 'pain knee', 'knee oa', 'oa knee', 'osteoarthrities knee', 'knee osteoarthritis', 
                      'kneepain', 'painknee', 'kneeoa', 'oaknee', 'osteoarthritiesknee', 'kneeosteoarthritis']
hip_oa_substrings = ['hip pain', 'pain hip', 'hip oa', 'oa hip', 'osteoarthrities hip', 'hip osteoarthritis',
                     'hippain', 'painhip', 'hipoa', 'oahip', 'osteoarthritieship', 'hiposteoarthritis']

# knee_oa_substrings = ['kneepain', 'painknee', 'kneeoa', 'oaknee', 'osteoarthritiesknee', 'kneeosteoarthritis']
# hip_oa_substrings = ['hip pain', 'pain hip', 'hip oa', 'oa hip', 'osteoarthrities hip', 'hip osteoarthritis',
#                      'hippain', 'painhip', 'hipoa', 'oahip', 'osteoarthritieship', 'hiposteoarthritis']

In [9]:
# This adds every individual word into a new list, and splits up strings that contain more than one word so that their
# individual words can be added to the list. The new list contains unique individual words that are found in any of the original
# strings. Returns a dictionary of individual words and their associated embeddings
def create_embeddings(substrings):  
    vectors = {}
    single_words =[]
    for i in substrings:
        if len(i.split()) == 1:
            single_words.append(i)
        else:
            two_words = i.split()
            for word in two_words:
                if word not in single_words:
                    single_words.append(word)
    for word in single_words:
        # print(word)
        try:
            word_array = bioword_vector[word]
        # If the word does not have an embedding already, we use the model to create one for it
        except:
            word_array = bioword_model.get_word_vector(word)
        vectors[word] = word_array
    return vectors

knee_oa_embeddings = create_embeddings(knee_oa_substrings)
hip_oa_embeddings = create_embeddings(hip_oa_substrings)

print(knee_oa_embeddings.keys())
print(hip_oa_embeddings.keys())
    
# print(np.sqrt(np.sum(np.square(knee_oa_embeddings['oa'] - hip_oa_embeddings['oa']))))

dict_keys(['kneepain', 'painknee', 'kneeoa', 'oaknee', 'osteoarthritiesknee', 'kneeosteoarthritis'])
dict_keys(['hip', 'pain', 'oa', 'osteoarthrities', 'osteoarthritis', 'hippain', 'painhip', 'hipoa', 'oahip', 'osteoarthritieship', 'hiposteoarthritis'])


In [10]:
# # Convert the word embeddings into CrypTen tensors, save the encrypted tensors to a file for future use
# def create_tensors(list_embeddings):
#     tensors = []
#     for key in list_embeddings:
#         # print(list_embeddings[key])
#         tensor = torch.Tensor(list_embeddings[key])
#         encrypted_tensor = crypten.cryptensor(tensor)
#         tensors.append(encrypted_tensor)
#     return tensors

# knee_oa_tensors = create_tensors(knee_oa_embeddings)
# hip_oa_tensors = create_tensors(hip_oa_embeddings)

# # print(knee_oa_tensors[0].get_plain_text())
# crypten.save(knee_oa_tensors, knee_tensors_path)
# crypten.save(hip_oa_tensors, hip_tensors_path)

In [11]:
# Convert the word embeddings into CrypTen tensors, save the encrypted tensors to a file for future use
def create_tensors(list_embeddings):
    tensors = []
    for key in list_embeddings:
        # print(list_embeddings[key])
        tensor = torch.Tensor(list_embeddings[key])
        # encrypted_tensor = crypten.cryptensor(tensor)
        # tensors.append(encrypted_tensor)
        tensors.append(tensor)
    return tensors

knee_oa_tensors = create_tensors(knee_oa_embeddings)
hip_oa_tensors = create_tensors(hip_oa_embeddings)


# print(knee_oa_tensors[0].get_plain_text())
# crypten.save(knee_oa_tensors, knee_tensors_path)
# crypten.save(hip_oa_tensors, hip_tensors_path)

/tmp/ipykernel_347/2350900036.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642881969/work/torch/csrc/utils/tensor_numpy.cpp:199.)
  tensor = torch.Tensor(list_embeddings[key])


In [12]:
# This function creates pickle files that can be retrieved later in the MPC protocol
def create_file(filename, tensors):
    with open(filename, 'wb') as f:
        torch.save(tensors, f)
    
create_file(knee_keywords_output, knee_oa_tensors)
create_file(hip_keywords_output, hip_oa_tensors)

In [13]:
# list_embeddings = []
# test_embedding = torch.Tensor(bioword_vector["osteoarthritis"])
# test_embedding2 = torch.Tensor(bioword_vector["osteoarthrities"])
# list_embeddings.append(test_embedding)
# list_embeddings.append(test_embedding2)
# # print(test_embedding)
# file_encrypted = "/home/jovyan/mpc_use_case/three_party_mpc/party2_structured/list_embedding.pt"
# torch.save(list_embeddings, file_encrypted)